In [115]:
import tweepy
import os
import pandas as pd
import yfinance as yf
import openai
import big_baller_moves
import json
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup

In [116]:
#Put your Bearer Token in the parenthesis below
client1 = tweepy.Client(bearer_token=big_baller_moves.bossman_tingz["twitter_api_key"])

# Authenticate
auth = tweepy.OAuth1UserHandler(big_baller_moves.bossman_tingz["consumer_key"], big_baller_moves.bossman_tingz["consumer_secret"], big_baller_moves.bossman_tingz["access_token"], 
                                big_baller_moves.bossman_tingz["access_token_secret"])

# Create API object
api = tweepy.API(auth)

# openai key
openai.api_key = big_baller_moves.bossman_tingz["openai_api_key"]

In [117]:
#enter username to get the user ID
username = "Heisenberg_100k"

try:
    user = client1.get_user(username=username)
    user_id = user_id = user.data.id
    print("User ID for", username, ":", user_id)
except Exception as e:
    print("Error:", e)

User ID for Heisenberg_100k : 1750349614786691072


In [118]:
# #get btc price (not working atm)

# # Choose Token and Time
# timestamp_open = pd.Timestamp('2024-05-02 05:09:00', tz='UTC')
# unix_timestamp = int(timestamp_open.timestamp())
# unix_timestamp_24 = unix_timestamp + (24*60*60)
# unix_timestamp_48 = unix_timestamp + (48*60*60)
# ckey = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp}&localization=false"
# url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"
# ckey24 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_24}&localization=false"
# ckey48 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_48}&localization=false"

# try:
#         # Convert timestamp to Unix timestamp (Kraken API requires timestamp in seconds)
#         unix_timestamp = int(timestamp.timestamp())

#         # Kraken API endpoint for OHLC data
#         url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"

# response = requests.get(url)
#         if response.status_code == 200:
#             data = response.json()
#             if 'result' in data and 'XBTUSD' in data['result'] and data['result']['XBTUSD']:
#                 # The response contains a list of lists, where each inner list represents OHLC data for a specific interval
#                 # We extract the closing price from the last list (most recent data point)
#                 bitcoin_price = float(data['result']['XBTUSD'][-1][4])
#                 return bitcoin_price
#             else:
#                 print("No data available for the specified timestamp")
#                 return None
#         else:
#             print(f"Failed to retrieve data from Kraken API. Status code: {response.status_code}")
#             return None
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return None


In [119]:
# unix_timestamp

In [120]:
#get all tweets
tweets = client1.get_users_tweets(
    id= user_id,
    max_results=100,  # Number of tweets to retrieve (adjust as needed)
    tweet_fields=['id', 'text', 'created_at', 'entities', 'attachments'],  # Fields you want to retrieve for each tweet
    media_fields=['preview_image_url', 'url'],
    exclude = ['retweets','replies'],
    expansions=['attachments.media_keys', 'author_id']
)


In [121]:
# Process retrieved tweets
# for tweet in tweets.data:
#     print(f"Tweet ID: {tweet.id}, Text: {tweet.text}, Created At: {tweet.created_at}")
df = pd.DataFrame(tweets.data)

In [122]:
#extract urls for tweets with images
def get_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        return display_urls
    else:
        return 0

# Apply the function to make the display_urls a column in our dataframe
df["image_url"] = df["entities"].apply(get_display_url)

selected_columns = ['id', 'text', 'created_at', 'image_url']

# Creating a new DataFrame with selected columns
heisenberg_tweets = df[selected_columns].copy()

In [123]:
heisenberg_tweets

,id,text,created_at,image_url
0,1791524527786332521,$VIX DCA’ed and pressed\n\nMy only position in the port for now.\n\n30 contracts at 3.03 average. It’s down $200 bucks no biggie.\n\nI’m looking for VIX to steady here next few weeks. Could enter 11 handle briefly. Then head to 15-16. Should get a decent return on these. Have 95 days. https://t.co/t2qfxLCjDy,2024-05-17 17:41:46+00:00,[pic.twitter.com/t2qfxLCjDy]
1,1791198206854148369,"For those who want to play the VIX but cannot because cough Robinhood cough, then I suggest SPY puts. It's pretty much the next best alternative.\n\nDon't give me that UVXY crap imo.",2024-05-16 20:05:06+00:00,0
2,1791197764661260308,"Since the April lows, the $SPX is up almost 8%. That is 19 trading days. Only 1 of those 19 was a scary day. The rest was mild reds and big green boners.\n\nI'm forecasting that we get another bout of 3-4% selloff here. Which is totally garden variety normal.\n\nWhen stocks go up 8%… https://t.co/IAWh2fPfjB",2024-05-16 20:03:20+00:00,[x.com/i/web/status/1…]
3,1791190695249838362,"$VIX Open\n\nRisk $4,650.\n\nHave 3+ months on these VIX calls.\n\nJust looking for 25-50% on these. Could take some time. https://t.co/9qf85HGxq0",2024-05-16 19:35:15+00:00,[pic.twitter.com/9qf85HGxq0]
4,1791189980225863994,"$CABA Closed\n\nIn 16.05 (4-19-24)\n\nOut 12.40\n\n-23%\n-$1,095 https://t.co/ZyqTD8by6K",2024-05-16 19:32:24+00:00,[pic.twitter.com/ZyqTD8by6K]
...,...,...,...,...
93,1785783922804281829,"I'm watching $NVDA, $AMD, and $SMCI closely for a bounce and only a bounce soon.\n\nNot yet.\n\nBut when we do, it will be a bagger. Could be tomorrow, could be Friday. I just need to see how much more they drop. Prefer one more big drop tomorrow.\n\nWait for my announcement if you… https://t.co/rOWppyUycl",2024-05-01 21:30:40+00:00,[x.com/i/web/status/1…]
94,1785681074993205606,$AMZN Closed\n\nIn .93c credit (yesterday)\n\nOut .08c debit\n\nNot worth holding the last couple days for the extra $40. Just in case. https://t.co/rfVfHbxQP4,2024-05-01 14:41:59+00:00,[pic.twitter.com/rfVfHbxQP4]
95,1785678634663207124,$SPY Open\n\nSelling these to pair with my call credit spreads.\n\nRisk $830 to make $170.\n\nThis was the lows from a couple weeks ago. https://t.co/hY0fpGwL19,2024-05-01 14:32:17+00:00,[pic.twitter.com/hY0fpGwL19]
96,1785451339046432869,This one is still open. Looking REALLY good now.,2024-04-30 23:29:05+00:00,0


In [124]:
#helper counting function to check display_urls (can ignore)

def count_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        return sum(1 for url in display_urls if url is not None)
    else:
        return 0

# Apply the function to count display_url
count = df["entities"].apply(count_display_url).sum()

print("Number of rows with display_url:", count)

Number of rows with display_url: 67


In [125]:
df[df['attachments'].isnull()]['entities']

1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     {'annotations': [{'start': 31, 'end': 33, 'probability': 0.6033, 'type': 'Other', 'normalized_text': 'VIX'}, {'start': 60, 'end': 68, 'probability': 0.5665, 'type': 'Organization', 'normalized_text': 'Robinhood'}, {'start': 92, 'end': 94, 'probability': 0.7388, 'type': 'Other', 'normalized

In [126]:
ticker_symbol = 'AAPL'

# Fetch historical data
stock_data = yf.Ticker(ticker_symbol)

# Get historical market data
historical_data = stock_data.history(period="max")

print(historical_data)

                                 Open        High         Low       Close  \
Date                                                                        
1980-12-12 00:00:00-05:00    0.099058    0.099488    0.099058    0.099058   
1980-12-15 00:00:00-05:00    0.094320    0.094320    0.093890    0.093890   
1980-12-16 00:00:00-05:00    0.087429    0.087429    0.086999    0.086999   
1980-12-17 00:00:00-05:00    0.089152    0.089582    0.089152    0.089152   
1980-12-18 00:00:00-05:00    0.091737    0.092167    0.091737    0.091737   
...                               ...         ...         ...         ...   
2024-05-13 00:00:00-04:00  185.440002  187.100006  184.619995  186.279999   
2024-05-14 00:00:00-04:00  187.509995  188.300003  186.289993  187.429993   
2024-05-15 00:00:00-04:00  187.910004  190.649994  187.369995  189.720001   
2024-05-16 00:00:00-04:00  190.470001  191.100006  189.660004  189.839996   
2024-05-17 00:00:00-04:00  189.380005  190.809998  189.220001  189.869995   

In [127]:
client = openai
df1 = df.copy().head(5)
pd.set_option('display.max_colwidth', None)

In [128]:
# #for astronomer
# system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [129]:
#heisenberg
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [130]:
def get_response(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Tell me if the message is suggesting opening a long, short or N/A in one word. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short." }, 
                                                        {"role":"user","content": text}])
   
    return response.choices[0].message.content.strip()



In [131]:
def get_response_guided(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Your job is to tag tweets as either opening a long position or short position or N/A. Provide a reason explaining why the tweet is a long or short or n/a." }, 
                                                        {"role":"user","content": "Taking profit"},
                                                        {"role":"assistant","content": "Short, the poster is taking profit."}])
   
    return response.choices[0].message.content.strip()


In [132]:
def get_response_image(text) :
    if text == 0:
        return "No image available"
    if not isinstance(text, str):
        print(text)
        text = text[0]
    try:
        response = client.chat.completions.create(model="gpt-4o", messages=[
                            {
                              "role": "user",
                              "content": [
                                {"type": "text", "text": "What kind of stock purchase is this image describing? If it is an option play like a call or a put please specify."},
                                {
                                  "type": "image_url",
                                  "image_url": {
                                    "url": text,
                                  },
                                },
                              ],
                            }
                          ], max_tokens=300,)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [133]:
def buy_or_sell_GPT(text) :
    try:
        response = client.chat.completions.create(model="gpt-4o", 
                                                   messages=[
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "What is this message and transcribed image data telling us? Please respond in format [Ticker (if any)] [Buy/Sell/Neither]. Only the ticker and Buy/Sell/"},
            {
              "type": "text",
              "text": text
            },
          ],
        }
      ],
      max_tokens=300,)
   
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [134]:
def buy_or_sell_GPT_k_shot(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": """Reference example as a guide to understand the format of the output.
Example:
Text: $PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. 
https://t.co/GpCKwDrfky 
TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. 
Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of $13.2401. 
It's a limit order set to sell at $13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.
Correct Output: [PARA] [Sell]

Text: If $META closes above 450, I will do 1,000 jumping jacks.\n\n🙏
TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]

Text: \\$GOOG Closed\n\nIn .32 credit (Apr 30th)\n\nOut .05 debit\n\n+\\$135 profit\n\nSmall. Got dangerously close. But never worried. TRANSCRIBED IMAGE DATA: The image describes a vertical options spread for Google (GOOG) stock, specifically a put spread. Here are the details:\n\n- **Type of Spread**: Vertical Put Spread\n- **Underlying Asset**: GOOG (Google)\n- **Expiry Date**: May 3, 2024\n- **Strike Prices**: 165 (bought) and 162.5 (sold)\n\nDetails of the trade:\n- **Quantity**: 5 contracts\n- **Total Cost**: Since you are paying 0.05 per contract for 5 contracts, the total debit is 5 * 0.05 = \\$25.\n\nIn this trade, you are buying 5 put options at a strike price of 165 and selling 5 put options at a strike price of 162.5 for the same expiration date of May 3, 2024. This creates a vertical spread (specifically a bear put spread), aiming to profit from a decrease in the price of the underlying stock, Google, down to or below the lower strike price of 162.5.
Correct Output: [GOOG] [Sell]

Text: \\$AMD Eyeing this name.\n\nIf it can get to 137-138ish tomorrow or next week, then I would buy there. \n\nWhy? 200dma magnet could act as major support.  TRANSCRIBED IMAGE DATA: None
Correct Output: [Neither]
"""
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "What is this message and transcribed image data telling us? Please respond in format [Ticker (if any)] [Buy/Sell/Neither]. If it is Neither, DO NOT put a ticker. Include no other information. Make sure the format is consistent, as in the examples provided in the system prompt."},
                        {"type": "text", "text": text}
                    ]
                }
            ],
            max_tokens=300,
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass


In [135]:
def dynamic_prompting(text, sys_prompt, user_prompt):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": sys_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": user_prompt},
                        {"type": "text", "text": text}
                    ]
                }
            ],
            max_tokens=300,
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass


In [136]:
# df1['Direction (Guided)'] = df1['text'].apply(get_response_guided)
# df1['Direction'] = df1['text'].apply(get_response)

In [137]:

heisenberg_tweets['Image'] = heisenberg_tweets['image_url'].apply(get_response_image)


['pic.twitter.com/t2qfxLCjDy']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/i/web/status/1…']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/9qf85HGxq0']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/ZyqTD8by6K']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/JHUGNI69Hk']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/TOm0oQWItb']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/1KT6Us0Ch

In [138]:

# selected = ['text','Direction','Direction (Guided)']
# df1[selected]

In [139]:
ht_mini = heisenberg_tweets

In [140]:
def initialize_webdriver():
    options = Options()
    options.add_argument("--headless")  # Run in background without opening a browser window
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    # Disable images, developer tools, pop-ups, extensions to speed up loading
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")  
    options.add_argument("--disable-popup-blocking") 
    options.add_argument("--disable-dev-tools")  
    options.add_argument('blink-settings=imagesEnabled=false')  

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def get_jpg_url(driver, url):
    try:
        if isinstance(url, list):  # Extract first element if URL is a list
            url = url[0] if url else None
        elif not url:
            return None
        if not url.startswith('http'):
            url = f'https://{url}'
        print(url)
        driver.get(url)
        # Wait for potential redirects and the page to stabilize
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'img')))
        time.sleep(1) # Short sleep to ensure images are loaded
        
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            img_src = img.get_attribute('src')
            if 'media' in img_src:
                return img_src

    except NoSuchElementException as e:
        print(f"Element not found for URL {url}: {e}")
    except TimeoutException as e:
        print(f"Request timed out for URL {url}: {e}")
    except Exception as e:
        print(f"General error processing URL {url}: {e}")
    return None

driver = initialize_webdriver()

try:
    ht_mini.loc[:, 'jpg_url'] = ht_mini['image_url'].apply(lambda x: get_jpg_url(driver, x))

finally:
    driver.quit()

https://pic.twitter.com/t2qfxLCjDy
https://x.com/i/web/status/1…
https://pic.twitter.com/9qf85HGxq0
https://pic.twitter.com/ZyqTD8by6K
https://pic.twitter.com/JHUGNI69Hk
https://pic.twitter.com/TOm0oQWItb
https://pic.twitter.com/1KT6Us0Chi
https://pic.twitter.com/ibhhyovQ9r
https://pic.twitter.com/AArc8zxr1M
https://pic.twitter.com/g3ur59gBxz
https://pic.twitter.com/GpCKwDrfky
https://pic.twitter.com/VUk3AXsCDY
https://pic.twitter.com/CceV23s8gb
https://pic.twitter.com/2G1TFQ8iaV
https://pic.twitter.com/LeXt1fPshM
https://pic.twitter.com/HRF5z5GmYK
https://x.com/i/web/status/1…
https://x.com/i/web/status/1…
https://pic.twitter.com/FJwJ8S1RdE
https://x.com/i/web/status/1…
https://pic.twitter.com/shmUd4IV7w
https://pic.twitter.com/1FrUXsjpbc
https://pic.twitter.com/FsZa8r2C56
https://pic.twitter.com/WNy2WINVwq
https://pic.twitter.com/wZl8ISwGxb
https://pic.twitter.com/iigSxnAU8S
https://pic.twitter.com/qsWN4xdihq
https://pic.twitter.com/rL3Y94Ex9E
https://x.com/i/web/status/1…
https://x.

In [141]:
ht_mini.head()

,id,text,created_at,image_url,Image,jpg_url
0,1791524527786332521,$VIX DCA’ed and pressed\n\nMy only position in the port for now.\n\n30 contracts at 3.03 average. It’s down $200 bucks no biggie.\n\nI’m looking for VIX to steady here next few weeks. Could enter 11 handle briefly. Then head to 15-16. Should get a decent return on these. Have 95 days. https://t.co/t2qfxLCjDy,2024-05-17 17:41:46+00:00,[pic.twitter.com/t2qfxLCjDy],None,https://pbs.twimg.com/media/GNzGD_vbsAAUgi7?format=jpg&name=small
1,1791198206854148369,"For those who want to play the VIX but cannot because cough Robinhood cough, then I suggest SPY puts. It's pretty much the next best alternative.\n\nDon't give me that UVXY crap imo.",2024-05-16 20:05:06+00:00,0,No image available,None
2,1791197764661260308,"Since the April lows, the $SPX is up almost 8%. That is 19 trading days. Only 1 of those 19 was a scary day. The rest was mild reds and big green boners.\n\nI'm forecasting that we get another bout of 3-4% selloff here. Which is totally garden variety normal.\n\nWhen stocks go up 8%… https://t.co/IAWh2fPfjB",2024-05-16 20:03:20+00:00,[x.com/i/web/status/1…],None,None
3,1791190695249838362,"$VIX Open\n\nRisk $4,650.\n\nHave 3+ months on these VIX calls.\n\nJust looking for 25-50% on these. Could take some time. https://t.co/9qf85HGxq0",2024-05-16 19:35:15+00:00,[pic.twitter.com/9qf85HGxq0],None,https://pbs.twimg.com/media/GNuWcVSboAAo90c?format=jpg&name=small
4,1791189980225863994,"$CABA Closed\n\nIn 16.05 (4-19-24)\n\nOut 12.40\n\n-23%\n-$1,095 https://t.co/ZyqTD8by6K",2024-05-16 19:32:24+00:00,[pic.twitter.com/ZyqTD8by6K],None,https://pbs.twimg.com/media/GNuVyzSbsAApokt?format=jpg&name=small


In [142]:
ht_mini.loc[:,'image_response'] = ht_mini['jpg_url'].apply(lambda x: get_response_image(x) if x != None else None)
ht_mini['image_response'] = ht_mini['image_response'].apply(lambda x: x.replace('$', '\$') if x and pd.notna(x) else x)
ht_mini['text'] = ht_mini['text'].apply(lambda x: x.replace('$', '\$') if pd.notna(x) else x)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:3: SyntaxWarning: invalid escape sequence '\$'
C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\2045728389.py:2: SyntaxWarning: invalid escape sequence '\$'
  ht_mini['image_response'] = ht_mini['image_response'].apply(lambda x: x.replace('$', '\$') if x and pd.notna(x) else x)
C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\2045728389.py:3: SyntaxWarning: invalid escape sequence '\$'
  ht_mini['text'] = ht_mini['text'].apply(lambda x: x.replace('$', '\$') if pd.notna(x) else x)


In [143]:
ht_mini['full_response'] = ht_mini['text'].astype(str) + '  TRANSCRIBED IMAGE DATA: ' + ht_mini['image_response'].astype(str)
ht_mini['buy_or_sell'] = ht_mini['full_response'].apply(buy_or_sell_GPT_k_shot)
ht_mini['buy_or_sell_2'] = ht_mini['full_response'].apply(buy_or_sell_GPT_k_shot)
ht_mini['buy_or_sell_3'] = ht_mini['full_response'].apply(buy_or_sell_GPT_k_shot)
ht_mini['all equal'] =  (ht_mini['buy_or_sell'] == ht_mini['buy_or_sell_2']) & (ht_mini['buy_or_sell_2'] == ht_mini['buy_or_sell_3'])

In [144]:
ht_mini['full_response'] = ht_mini['text'].astype(str) + '  TRANSCRIBED IMAGE DATA: ' + ht_mini['image_response'].astype(str)
ht_mini['buy_or_sell'] = ht_mini['full_response'].apply(buy_or_sell_GPT)
ht_mini['buy_or_sell_2'] = ht_mini['full_response'].apply(buy_or_sell_GPT)
ht_mini['buy_or_sell_3'] = ht_mini['full_response'].apply(buy_or_sell_GPT)
ht_mini['all equal'] =  (ht_mini['buy_or_sell'] == ht_mini['buy_or_sell_2']) & (ht_mini['buy_or_sell_2'] == ht_mini['buy_or_sell_3'])

In [145]:
ht_mini['2-3 equal'] =   (ht_mini['buy_or_sell_2'] == ht_mini['buy_or_sell_3'])

In [146]:
ht_mini['buy_or_sell_4'] = ht_mini['full_response'].apply(buy_or_sell_GPT_k_shot)
ht_mini['buy_or_sell_5'] = ht_mini['full_response'].apply(buy_or_sell_GPT_k_shot)
ht_mini['buy_or_sell_6'] = ht_mini['full_response'].apply(buy_or_sell_GPT_k_shot)
ht_mini['rnd 2 all equal'] =  (ht_mini['buy_or_sell_4'] == ht_mini['buy_or_sell_5']) & (ht_mini['buy_or_sell_5'] == ht_mini['buy_or_sell_6'])

In [72]:
display(ht_mini)

,id,text,created_at,image_url,Image,jpg_url,image_response,full_response,buy_or_sell,buy_or_sell_2,buy_or_sell_3,all equal,2-3 equal
0,1790443819621924935,Sony rethinking their bid. PARA down a 1 pointt. K got lucky there.,2024-05-14 18:07:26+00:00,0,No image available,None,None,Sony rethinking their bid. PARA down a 1 pointt. K got lucky there. TRANSCRIBED IMAGE DATA: None,[PARA] [Neither],[PARA] [Neither],[Neither],False,False
1,1790421057742205068,\$PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+\$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky,2024-05-14 16:36:59+00:00,[pic.twitter.com/GpCKwDrfky],None,https://pbs.twimg.com/media/GNjadkgacAAB0Gy?format=jpg&name=small,"This image describes a stock sale transaction, not an option play like a call or a put. Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of \$13.2401. It's a limit order set to sell at \$13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.","\$PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+\$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of \$13.2401. It's a limit order set to sell at \$13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.",[PARA] [Sell],[PARA] [Sell],[PARA] [Sell],True,True
2,1790415982328263151,"\$INTC Closed\n\nIn 30.53 (yesterday)\n\nOut 30.91\n\n+1.3%\n+\$152 profit\n\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY",2024-05-14 16:16:49+00:00,[pic.twitter.com/VUk3AXsCDY],None,https://pbs.twimg.com/media/GNjV1NsbwAAwYeB?format=jpg&name=small,"The image describes a stock trade involving shares of INTC, where 400 shares were sold at a limit price of \$30.91. This is indicated by the ""SELL @30.91 LMT"" and the series of completed trade fills with negative quantities. This is not related to options trading (such as calls or puts) but rather the direct selling of stock shares.","\$INTC Closed\n\nIn 30.53 (yesterday)\n\nOut 30.91\n\n+1.3%\n+\$152 profit\n\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY TRANSCRIBED IMAGE DATA: The image describes a stock trade involving shares of INTC, where 400 shares were sold at a limit price of \$30.91. This is indicated by the ""SELL @30.91 LMT"" and the series of completed trade fills with negative quantities. This is not related to options trading (such as calls or puts) but rather the direct selling of stock shares.",[INTC] [Sell],[INTC] [Sell],[INTC] [Sell],True,True
3,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,2024-05-13 17:51:30+00:00,0,No image available,None,None,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad? TRANSCRIBED IMAGE DATA: None,[GME] [Sell],[GME] [Sell],[GME] [Sell],True,True
4,1790077076105498901,\$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07+00:00,[pic.twitter.com/CceV23s8gb],None,https://pbs.twimg.com/media/GNehnPvbMAAq-N7?format=jpg&name=small,"The image describes a purchase of stock, not an option play. Here are the details:\n\n- **Stock**: PARA\n- **Quantity**: 500 shares\n- **Price Per Share**: \$13.1099\n- **Limit Order Price**: \$13.11\n\nThe order was executed on 5/13/24 at 10:46 AM. The term ""LMT"" refers to a limit order, which means the shares were purchased at a price of \$13.11 or better. \n\nThis indicates a straightforward stock purchase, not a call or put option trade.","\$PAR

In [83]:
selected_cols = ['created_at', 'full_response', 'buy_or_sell','buy_or_sell_2','buy_or_sell_3','all equal',
                 '2-3 equal','buy_or_sell_4','buy_or_sell_5','buy_or_sell_6','rnd 2 all equal']
ht_fin = ht_mini[selected_cols]
cwd = os.getcwd()
file_path = os.path.join(cwd, 'tweet_buy_or_sell_3.xlsx')
ht_fin['created_at'] = ht_fin['created_at'].dt.tz_localize(None)
ht_fin = ht_fin.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
print(file_path)
ht_fin.to_excel(file_path, index=False)

C:\Users\abdan\Project_X\tweet_buy_or_sell_3.xlsx


C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\1453288636.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ht_fin['created_at'] = ht_fin['created_at'].dt.tz_localize(None)
C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\1453288636.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ht_fin = ht_fin.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)


In [147]:
#I've separated the prompting into two separate columns, one to get the ticker (here) and another to get the direction (below)

cols = ['id', 'created_at', 'full_response']
ht_dynamic = ht_mini[cols]
sys_prompt = """Reference example as a guide to understand the format of the output.
Example:
Text: $PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. 
https://t.co/GpCKwDrfky 
TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. 
Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of $13.2401. 
It's a limit order set to sell at $13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.
Correct Output: PARA

Text: If $META closes above 450, I will do 1,000 jumping jacks.\n\n🙏
TRANSCRIBED IMAGE DATA: None
Correct Output: META

Text: \\$GOOG Closed\n\nIn .32 credit (Apr 30th)\n\nOut .05 debit\n\n+\\$135 profit\n\nSmall. Got dangerously close. But never worried. TRANSCRIBED IMAGE DATA: The image describes a vertical options spread for Google (GOOG) stock, specifically a put spread. Here are the details:\n\n- **Type of Spread**: Vertical Put Spread\n- **Underlying Asset**: GOOG (Google)\n- **Expiry Date**: May 3, 2024\n- **Strike Prices**: 165 (bought) and 162.5 (sold)\n\nDetails of the trade:\n- **Quantity**: 5 contracts\n- **Total Cost**: Since you are paying 0.05 per contract for 5 contracts, the total debit is 5 * 0.05 = \\$25.\n\nIn this trade, you are buying 5 put options at a strike price of 165 and selling 5 put options at a strike price of 162.5 for the same expiration date of May 3, 2024. This creates a vertical spread (specifically a bear put spread), aiming to profit from a decrease in the price of the underlying stock, Google, down to or below the lower strike price of 162.5.
Correct Output: GOOG

Text: \\$AMD Eyeing this name.\n\nIf it can get to 137-138ish tomorrow or next week, then I would buy there. \n\nWhy? 200dma magnet could act as major support.  TRANSCRIBED IMAGE DATA: None
Correct Output: AMD

Text: \\$DIS Open\n\nEarnings play\n\nRisk \\$1,670 to make \\$330.\n\nBmo, implied about a 7pt move. https://t.co/nyMZGS23OZ  TRANSCRIBED IMAGE DATA: The image describes a vertical put spread (also known as a bull put spread). This is a type of options strategy that involves selling one put option and buying another put option at a lower strike price but with the same expiration date.\n\nHere's the breakdown of the trade based on the image:\n\n- This is for the stock with the ticker symbol "DIS" (Walt Disney Company), with options expiring on May 10, 2024.\n- The vertical spread involves the 108 and 106 strike prices, meaning you are dealing with 108/106 put options.\n- The strategy specified is a put vertical spread:\n  - Selling 10 put options at the 108 strike price.\n  - Buying 10 put options at the 106 strike price.\n- The prices for the transactions are:\n  - Sold (shorted) the 108 strike put options at \\$0.86 each.\n  - Bought (long) the 106 strike put options at \\$0.53 each.\n- The net credit received for the spread is \\$0.33 per share (since options typically represent 100 shares, the total net credit is \\$33 per contract).\n\nIn summary, the strategy is a bull put spread where you hope the price of Disney (DIS) stays above the higher strike price (108) by the expiration date so that both options expire worthless, and you keep the credit received upfront.
Correct Output: DIS

Text: Watchlist for next week:\n\nLong: \\$VIX\n\nShort: \\$FXI, \\$SPY\n\nNeutral: \\$NVDA\n\nSpeculative bounce play on watch: \\$SHOP  TRANSCRIBED IMAGE DATA: None
Correct Output: VIX, FXI, SPY, NVDA, SHOP

Text: Two Biggest Losers this week:\n\nSPY call credit spread: -\\$5,130\n\nDIS put credit spread: -\\$1,670\n\nOuch.\n\nAccount Balance: \\$114,888.39.\n\n-\\$5,097.20 week over week.\n\nNot happy, but we look forward to next week. https://t.co/X67fSZOUY6  TRANSCRIBED IMAGE DATA: None
Correct Output: SPY, DIS
"""
user_prompt = "What stock is this message and transcribed image data referring to? Please respond with just the Ticker. If the tweet refers to multiple tickers, list them all in a comma separated list."
ht_dynamic['ticker'] = ht_mini['full_response'].apply(
    lambda text: dynamic_prompting(text, sys_prompt, user_prompt)
)
ht_dynamic['ticker_2'] = ht_mini['full_response'].apply(
    lambda text: dynamic_prompting(text, sys_prompt, user_prompt)
)

C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\4133358789.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ht_dynamic['ticker'] = ht_mini['full_response'].apply(
C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\4133358789.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ht_dynamic['ticker_2'] = ht_mini['full_response'].apply(


In [148]:
user_prompt = "What direction is this message and transcribed image data referring to? Please respond with just Buy, Sell, or Neither. If the tweet refers to multiple tickers, list the direction for each in a comma separated list."
sys_prompt = """Reference example as a guide to understand the format of the output.
Example:
Text: $PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. 
https://t.co/GpCKwDrfky 
TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. 
Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of $13.2401. 
It's a limit order set to sell at $13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.
Correct Output: Sell

Text: If $META closes above 450, I will do 1,000 jumping jacks.\n\n🙏
TRANSCRIBED IMAGE DATA: None
Correct Output: Neither

Text: \\$GOOG Closed\n\nIn .32 credit (Apr 30th)\n\nOut .05 debit\n\n+\\$135 profit\n\nSmall. Got dangerously close. But never worried. TRANSCRIBED IMAGE DATA: The image describes a vertical options spread for Google (GOOG) stock, specifically a put spread. Here are the details:\n\n- **Type of Spread**: Vertical Put Spread\n- **Underlying Asset**: GOOG (Google)\n- **Expiry Date**: May 3, 2024\n- **Strike Prices**: 165 (bought) and 162.5 (sold)\n\nDetails of the trade:\n- **Quantity**: 5 contracts\n- **Total Cost**: Since you are paying 0.05 per contract for 5 contracts, the total debit is 5 * 0.05 = \\$25.\n\nIn this trade, you are buying 5 put options at a strike price of 165 and selling 5 put options at a strike price of 162.5 for the same expiration date of May 3, 2024. This creates a vertical spread (specifically a bear put spread), aiming to profit from a decrease in the price of the underlying stock, Google, down to or below the lower strike price of 162.5.
Correct Output: Sell

Text: \\$AMD Eyeing this name.\n\nIf it can get to 137-138ish tomorrow or next week, then I would buy there. \n\nWhy? 200dma magnet could act as major support.  TRANSCRIBED IMAGE DATA: None
Correct Output: Neither

Text: \\$DIS Open\n\nEarnings play\n\nRisk \\$1,670 to make \\$330.\n\nBmo, implied about a 7pt move. https://t.co/nyMZGS23OZ  TRANSCRIBED IMAGE DATA: The image describes a vertical put spread (also known as a bull put spread). This is a type of options strategy that involves selling one put option and buying another put option at a lower strike price but with the same expiration date.\n\nHere's the breakdown of the trade based on the image:\n\n- This is for the stock with the ticker symbol "DIS" (Walt Disney Company), with options expiring on May 10, 2024.\n- The vertical spread involves the 108 and 106 strike prices, meaning you are dealing with 108/106 put options.\n- The strategy specified is a put vertical spread:\n  - Selling 10 put options at the 108 strike price.\n  - Buying 10 put options at the 106 strike price.\n- The prices for the transactions are:\n  - Sold (shorted) the 108 strike put options at \\$0.86 each.\n  - Bought (long) the 106 strike put options at \\$0.53 each.\n- The net credit received for the spread is \\$0.33 per share (since options typically represent 100 shares, the total net credit is \\$33 per contract).\n\nIn summary, the strategy is a bull put spread where you hope the price of Disney (DIS) stays above the higher strike price (108) by the expiration date so that both options expire worthless, and you keep the credit received upfront.
Correct Output: Buy

Text: Watchlist for next week:\n\nLong: \\$VIX\n\nShort: \\$FXI, \\$SPY\n\nNeutral: \\$NVDA\n\nSpeculative bounce play on watch: \\$SHOP  TRANSCRIBED IMAGE DATA: None
Correct Output: Buy, Sell, Sell, Neither, Neither

Text: Two Biggest Losers this week:\n\nSPY call credit spread: -\\$5,130\n\nDIS put credit spread: -\\$1,670\n\nOuch.\n\nAccount Balance: \\$114,888.39.\n\n-\\$5,097.20 week over week.\n\nNot happy, but we look forward to next week. https://t.co/X67fSZOUY6  TRANSCRIBED IMAGE DATA: None
Correct Output: Neither, Neither
"""
ht_dynamic['direction'] = ht_mini['full_response'].apply(
    lambda text: dynamic_prompting(text, sys_prompt, user_prompt)
)
ht_dynamic['direction_2'] = ht_mini['full_response'].apply(
    lambda text: dynamic_prompting(text, sys_prompt, user_prompt)
)

C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\1414707631.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ht_dynamic['direction'] = ht_mini['full_response'].apply(
C:\Users\abdan\AppData\Local\Temp\ipykernel_5092\1414707631.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ht_dynamic['direction_2'] = ht_mini['full_response'].apply(


In [155]:
def prompt_checking_buy(text_in, text_out):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": """Reference this example as a guide to understand the format of what is being received. Example: Input: """ + text_in + """ Output:""" + text_out +
                    """Reference the following examples to understand how to answer the question of what direction this option play is: Text: $PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. 
https://t.co/GpCKwDrfky 
TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. 
Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of $13.2401. 
It's a limit order set to sell at $13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.
Correct Output: Sell

Text: If $META closes above 450, I will do 1,000 jumping jacks.\n\n🙏
TRANSCRIBED IMAGE DATA: None
Correct Output: Neither

Text: \\$GOOG Closed\n\nIn .32 credit (Apr 30th)\n\nOut .05 debit\n\n+\\$135 profit\n\nSmall. Got dangerously close. But never worried. TRANSCRIBED IMAGE DATA: The image describes a vertical options spread for Google (GOOG) stock, specifically a put spread. Here are the details:\n\n- **Type of Spread**: Vertical Put Spread\n- **Underlying Asset**: GOOG (Google)\n- **Expiry Date**: May 3, 2024\n- **Strike Prices**: 165 (bought) and 162.5 (sold)\n\nDetails of the trade:\n- **Quantity**: 5 contracts\n- **Total Cost**: Since you are paying 0.05 per contract for 5 contracts, the total debit is 5 * 0.05 = \\$25.\n\nIn this trade, you are buying 5 put options at a strike price of 165 and selling 5 put options at a strike price of 162.5 for the same expiration date of May 3, 2024. This creates a vertical spread (specifically a bear put spread), aiming to profit from a decrease in the price of the underlying stock, Google, down to or below the lower strike price of 162.5.
Correct Output: Sell

Text: \\$AMD Eyeing this name.\n\nIf it can get to 137-138ish tomorrow or next week, then I would buy there. \n\nWhy? 200dma magnet could act as major support.  TRANSCRIBED IMAGE DATA: None
Correct Output: Neither

Text: \\$DIS Open\n\nEarnings play\n\nRisk \\$1,670 to make \\$330.\n\nBmo, implied about a 7pt move. https://t.co/nyMZGS23OZ  TRANSCRIBED IMAGE DATA: The image describes a vertical put spread (also known as a bull put spread). This is a type of options strategy that involves selling one put option and buying another put option at a lower strike price but with the same expiration date.\n\nHere's the breakdown of the trade based on the image:\n\n- This is for the stock with the ticker symbol "DIS" (Walt Disney Company), with options expiring on May 10, 2024.\n- The vertical spread involves the 108 and 106 strike prices, meaning you are dealing with 108/106 put options.\n- The strategy specified is a put vertical spread:\n  - Selling 10 put options at the 108 strike price.\n  - Buying 10 put options at the 106 strike price.\n- The prices for the transactions are:\n  - Sold (shorted) the 108 strike put options at \\$0.86 each.\n  - Bought (long) the 106 strike put options at \\$0.53 each.\n- The net credit received for the spread is \\$0.33 per share (since options typically represent 100 shares, the total net credit is \\$33 per contract).\n\nIn summary, the strategy is a bull put spread where you hope the price of Disney (DIS) stays above the higher strike price (108) by the expiration date so that both options expire worthless, and you keep the credit received upfront.
Correct Output: Buy

Text: Watchlist for next week:\n\nLong: \\$VIX\n\nShort: \\$FXI, \\$SPY\n\nNeutral: \\$NVDA\n\nSpeculative bounce play on watch: \\$SHOP  TRANSCRIBED IMAGE DATA: None
Correct Output: Buy, Sell, Sell, Neither, Neither

Text: Two Biggest Losers this week:\n\nSPY call credit spread: -\\$5,130\n\nDIS put credit spread: -\\$1,670\n\nOuch.\n\nAccount Balance: \\$114,888.39.\n\n-\\$5,097.20 week over week.\n\nNot happy, but we look forward to next week. https://t.co/X67fSZOUY6  TRANSCRIBED IMAGE DATA: None
Correct Output: Neither, Neither"""
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Based on the input and output provided in the system, take another look at the input and check if the output value is correct. THe instructions provided in the previously GPT-generated column that the output is from had these instructions: What direction is this message and transcribed image data referring to? Please respond with just Buy, Sell, or Neither. If the tweet refers to multiple tickers, list the direction for each in a comma separated list." },
                        {"type": "text", "text": text_in}
                    ]
                }
            ],
            max_tokens=300,
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass


In [158]:
def prompt_checking_ticker(text_in, text_out):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "system",
                    "content": """Reference example as a guide to understand the format of what is being received. Example: Input: """ + text_in + """ Output:""" + text_out
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Based on the input and output provided in the system, take another look at the input and check if the output value is correct. THe instructions provided in the previously GPT-generated column that the output is from had these instructions: What ticker is this message and transcribed image data referring to? Please respond in the one word format: Ticker. If the tweet refers to multiple tickers, list all in a comma separated list." },
                        {"type": "text", "text": text_in}
                    ]
                }
            ],
            max_tokens=300,
        )
        
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass


In [108]:
display(ht_dynamic)

,id,created_at,full_response,ticker,direction
0,1790443819621924935,2024-05-14 18:07:26,Sony rethinking their bid. PARA down a 1 pointt. K got lucky there. TRANSCRIBED IMAGE DATA: None,PARA,Neither
1,1790421057742205068,2024-05-14 16:36:59,"\\$PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+\\$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky TRANSCRIBED IMAGE DATA: This image describes a stock sale transaction, not an option play like a call or a put. Specifically, it details the sale of 500 shares of PARA (which is the ticker symbol for a stock) at an average fill price of \\$13.2401. It's a limit order set to sell at \\$13.24. The negative quantity (-500) indicates that shares are being sold rather than purchased.",PARA,Sell
2,1790415982328263151,2024-05-14 16:16:49,"\\$INTC Closed\n\nIn 30.53 (yesterday)\n\nOut 30.91\n\n+1.3%\n+\\$152 profit\n\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY TRANSCRIBED IMAGE DATA: The image describes a stock trade involving shares of INTC, where 400 shares were sold at a limit price of \\$30.91. This is indicated by the ""SELL @30.91 LMT"" and the series of completed trade fills with negative quantities. This is not related to options trading (such as calls or puts) but rather the direct selling of stock shares.",INTC,Sell
3,1790077422697693598,2024-05-13 17:51:30,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn\u2019t be too bad? TRANSCRIBED IMAGE DATA: None,GME,Sell
4,1790077076105498901,2024-05-13 17:50:07,"\\$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb TRANSCRIBED IMAGE DATA: The image describes a purchase of stock, not an option play. Here are the details:\n\n- **Stock**: PARA\n- **Quantity**: 500 shares\n- **Price Per Share**: \\$13.1099\n- **Limit Order Price**: \\$13.11\n\nThe order was executed on 5/13/24 at 10:46 AM. The term ""LMT"" refers to a limit order, which means the shares were purchased at a price of \\$13.11 or better. \n\nThis indicates a straightforward stock purchase, not a call or put option trade.",PARA,Buy
...,...,...,...,...,...
93,1783943988334174681,2024-04-26 19:39:25,"If \\$META closes above 450, I will do 1,000 jumping jacks.\n\n\U0001f64f TRANSCRIBED IMAGE DATA: None",META,Neither
94,1783943747711148219,2024-04-26 19:38:28,"\\$SPY Open\n\nRisk \\$1,750 to make \\$750.\n\nI\u2019m looking for some consolidation with a little downside bias to next week.\n\nThink SPY max upside is probably 512-513. Downside probably 500-505.\n\nDon\u2019t matter just stay below 515. https://t.co/OjXQoj4O34 TRANSCRIBED IMAGE DATA: The image describes a vertical spread option trade. Specifically, it is a ""Vertical Call Spread"" for SPY (S&P 500 ETF) expiring on May 3, 2024.\n\nHere's the breakdown:\n\n1. **Vertical Call Spread:**\n - A vertical spread involves buying and selling two options of the same type (calls or puts) with the same expiration date but different strike prices.\n - The trader sells 25 call options with a strike price of 515 and simultaneously buys 25 call options with a strike price of 516.\n\n2. **Trade details:**\n - For the call options sold (strike price of 515), 25 contracts were sold at \\$1.70.\n - For the call options bought (strike price of 516), 25 contracts were bought at \\$1.40.\n - The trader received a net credit of \\$0.30 per contract (\\$1.70 - \\$1.40).\n\n3. **Net position:**\n - Since the average fill price is -25 @ 0.30, this indicates the trader received a net credit of \\$0.30 per spread.\n\nIn summary, this image details a credit vertical call spread where the trader is betting that the price of SPY will be below 515 at expiration to keep the credit received.",SPY,Sell
95,1783941606837784711,2024-04-26 19:29:57,\\$SPY credit spread to expire OTM.\n\n\u2764\ufe0f if you followed. https://t.co/cxHfzEp0Xt TRANSCRIBED IMAGE DATA: None,SPY,Neither
96,17

In [156]:
ht_dynamic['proofread buy'] = ht_dynamic.apply(
    lambda row: prompt_checking_buy(row['full_response'], row['direction']), axis=1
)

In [159]:
ht_dynamic['proofread ticker'] = ht_dynamic.apply(
    lambda row: prompt_checking_ticker(row['full_response'], row['ticker']), axis=1
)

In [153]:
display(ht_dynamic)

,id,created_at,full_response,ticker,ticker_2,direction,direction_2,proofread buy,proofread ticker
0,1791524527786332521,2024-05-17 17:41:46+00:00,"\$VIX DCA’ed and pressed\n\nMy only position in the port for now.\n\n30 contracts at 3.03 average. It’s down \$200 bucks no biggie.\n\nI’m looking for VIX to steady here next few weeks. Could enter 11 handle briefly. Then head to 15-16. Should get a decent return on these. Have 95 days. https://t.co/t2qfxLCjDy TRANSCRIBED IMAGE DATA: The image describes a stock options trade, specifically the purchase of call options:\n\n- **VIX 100**: The underlying asset is likely related to the VIX index.\n- **21 AUG 24 [AM] (95)**: The expiration date of the options is August 21, 2024, with likely morning expiration.\n- **12.5 C**: This indicates a call option with a strike price of 12.5.\n- **+15 BUY @ 2.95 LMT**: The trader has bought 15 call options at a limit price of \$2.95.\n\nSo, this is a purchase of 15 call options at a strike price of 12.5, expiring on August 21, 2024, at a price of \$2.95 per option.",VIX,VIX,Buy,Buy,Buy,The ticker referred to in the message and the transcribed image data is:\n\nVIX
1,1791198206854148369,2024-05-16 20:05:06+00:00,"For those who want to play the VIX but cannot because cough Robinhood cough, then I suggest SPY puts. It's pretty much the next best alternative.\n\nDon't give me that UVXY crap imo. TRANSCRIBED IMAGE DATA: None","SPY, UVXY","SPY, UVXY",Sell,Buy,Sell,"The output is correct.\n\nInput: For those who want to play the VIX but cannot because cough Robinhood cough, then I suggest SPY puts. It's pretty much the next best alternative.\n\nDon't give me that UVXY crap imo. TRANSCRIBED IMAGE DATA: None\n\nOutput: SPY, UVXY"
2,1791197764661260308,2024-05-16 20:03:20+00:00,"Since the April lows, the \$SPX is up almost 8%. That is 19 trading days. Only 1 of those 19 was a scary day. The rest was mild reds and big green boners.\n\nI'm forecasting that we get another bout of 3-4% selloff here. Which is totally garden variety normal.\n\nWhen stocks go up 8%… https://t.co/IAWh2fPfjB TRANSCRIBED IMAGE DATA: None",SPX,SPX,Neither,Neither,Sell,SPX
3,1791190695249838362,2024-05-16 19:35:15+00:00,"\$VIX Open\n\nRisk \$4,650.\n\nHave 3+ months on these VIX calls.\n\nJust looking for 25-50% on these. Could take some time. https://t.co/9qf85HGxq0 TRANSCRIBED IMAGE DATA: This image describes a purchase of call options. Specifically, it shows the purchase of 15 call options contracts for VIX 100 with a strike price of 12.5, expiring on August 24th. The call options were bought at a limit price of 3.10.",VIX,VIX,Buy,Buy,Buy,"The provided output is correct. The tweet and transcribed image data refer to the ticker ""VIX""."
4,1791189980225863994,2024-05-16 19:32:24+00:00,"\$CABA Closed\n\nIn 16.05 (4-19-24)\n\nOut 12.40\n\n-23%\n-\$1,095 https://t.co/ZyqTD8by6K TRANSCRIBED IMAGE DATA: The image describes a transaction for CABA stock where 300 shares have been sold at a limit price of \$12.40. This is a straightforward stock sale transaction and not an options play such as a call or a put. The transaction details include:\n\n- Quantity (Qty): 300 shares sold (indicated by the negative sign).\n- Price: \$12.4046 per share.\n- Average Fill Price: -300 @ \$12.4046, indicating the average price per share for the sell order.\n\nSince this is about a direct sale of shares, it doesn’t involve purchasing or selling options.",CABA,CABA,Sell,Sell,Sell,"The output ticker ""CABA"" is correct as per the provided input and transcribed image data. The transaction and details mentioned are all related to the ticker ""CABA"". No multiple tickers are mentioned in the input. Therefore, the output value correctly reflects the single ticker ""CABA""."
...,...,...,...,...,...,...,...,...,...
93,1785783922804281829,2024-05-01 21:30:40+00:00,"I'm watching \$NVDA, \$AMD, and \$SMCI closely for a bounce and only a bounce soon.\n\nNot yet.\n\nBut when we do, it will be a bagger. Could be tomorrow, c

In [160]:
cwd = os.getcwd()
file_path = os.path.join(cwd, 'tweet_dynamic_multiple.xlsx')
ht_dynamic['created_at'] = ht_dynamic['created_at'].dt.tz_localize(None)
ht_dynamic = ht_dynamic.map(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
print(file_path)
ht_dynamic.to_excel(file_path, index=False)

C:\Users\abdan\Project_X\tweet_dynamic_multiple.xlsx
